# KEGG 化合物生成自由能计算

本项目用于计算 KEGG 数据库中前100个化合物，在 pH=9 条件下的标准生成吉布斯自由能（ΔG'f）。

**方法**: 组件贡献法 (Component Contribution, CC)，通过 `equilibrator_api` 实现。

In [1]:
import pandas as pd
from equilibrator_api import ComponentContribution, Q_
from tqdm.notebook import tqdm
import numpy as np
import warnings

warnings.filterwarnings('ignore')

## 1. 初始化和参数设置

在进行计算前，我们首先需要：
1. 初始化 `ComponentContribution` 对象。这个过程可能会比较慢，我们只执行一次。
2. 设置热力学条件：pH, pMg, 离子强度, 温度。

In [2]:
print("正在初始化 ComponentContribution，请稍候...")
cc = ComponentContribution()
print("初始化完成。")

# 设置用户指定的条件
cc.p_h = Q_(9.0)
cc.p_mg = Q_(3.0)  # 默认值
cc.ionic_strength = Q_('0.25M') # 默认值
cc.temperature = Q_('298.15K') # 默认值

print(f"计算条件已设置为: pH={cc.p_h.m}, pMg={cc.p_mg.m}, I={cc.ionic_strength.m}, T={cc.temperature.m}")

正在初始化 ComponentContribution，请稍候...
初始化完成。
计算条件已设置为: pH=9.0, pMg=3.0, I=0.25, T=298.15


## 2. 生成化合物ID并进行计算

我们生成 KEGG 化合物 ID 列表（C00001 到 C00100），然后遍历列表，计算每个化合物的 ΔG'f。

In [5]:
kegg_ids = [f"C{str(i).zfill(5)}" for i in range(1, 101)]
results = []

print(f"开始为 {len(kegg_ids)} 个化合物计算生成自由能...")

for kegg_id in tqdm(kegg_ids):
    try:
        # 尝试通过kegg id获取化合物对象
        compound = cc.get_compound(f"kegg:{kegg_id}")
        
        if compound is None:
            results.append({
                "KEGG_ID": kegg_id,
                "dGf_prime (kJ/mol)": np.nan,
                "Uncertainty (kJ/mol)": np.nan,
                "Error": "Compound not found in Equilibrator database"
            })
            continue
        
        # 计算标准生成自由能
        standard_dgf_prime, sigma_fin, sigma_inf = cc.standard_dg_formation(compound)
        
        # 获取数值和不确定度
        dg_value = standard_dgf_prime
        uncertainty = np.linalg.norm(sigma_fin) if sigma_fin is not None else 0.0
        
        results.append({
            "KEGG_ID": kegg_id,
            "dGf_prime (kJ/mol)": dg_value,
            "Uncertainty (kJ/mol)": uncertainty,
            "Error": "" 
        })

    except Exception as e:
        results.append({
            "KEGG_ID": kegg_id,
            "dGf_prime (kJ/mol)": np.nan,
            "Uncertainty (kJ/mol)": np.nan,
            "Error": str(e)
        })

print("计算完成。")

开始为 100 个化合物计算生成自由能...


  0%|          | 0/100 [00:00<?, ?it/s]

计算完成。


## 3. 保存结果

将计算结果整理成 DataFrame 并保存为 CSV 文件。

In [6]:
results_df = pd.DataFrame(results)
output_path = 'kegg_cc_ph9_results.csv'
results_df.to_csv(output_path, index=False)

print(f"结果已保存到: {output_path}")
display(results_df.head())

结果已保存到: kegg_cc_ph9_results.csv


,KEGG_ID,dGf_prime (kJ/mol),Uncertainty (kJ/mol),Error
0,C00001,-238.136712,0.759034,
1,C00002,-2811.578332,1.489986,
2,C00003,-2261.380648,6.693914,
3,C00004,-2236.551670,6.697240,
4,C00005,-3070.526029,6.829013,
